In [22]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences # to add padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding # to get word embeddedings

reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0]) # first 5 are positive reviews and, next 5 are negetive reviews.

In [23]:
# this is basically one-hot encoding of vectors.
# Convert the words in the text into one-hot vectors using a vocabulary size of 30.
one_hot("amazing restaurant",30)

# Splits the text into words, Applies a hashing function to each word
# Maps each word to an integer index in the range 1 to 30 in this case
# in this case it returned 3 for the word 'amazing' and 2 for the word 'restaurant'
# internally these will be converted to a vector with only 0 and 1.

[3, 2]

In [24]:
# the above code is only for that 2 words 'amazing' and 'restaurant'
# now i wanna do the same for entire reviews list.
vocab_size = 30
encoded_reviews = [one_hot(d, vocab_size) for d in reviews]
print(encoded_reviews)

# now this encoded unique values to every words in all reviews. internally coverted them to vectors with only 0 and 1.

[[6, 21], [3, 2], [23, 13], [21, 9, 12], [4, 16, 17], [7, 21], [21, 16, 13], [27, 7], [27, 23], [20, 19]]


In [25]:
# as you can see, some reviews are 3 words long and some are 2 words long
# so we need to add padding (append zeros at the end) to the review containing 2 words or less.
# this is done to match the length of all words.
# here we are just coverting all reviews to reviews containing 4 words each.
# i can also give max_length to 3 as well, as my max number of words in each review is 3.
max_length = 4
# passing our one-hot encoded vectors (we modified above) to pad_sequences() function
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post') # padding='post' means append in the end
print(padded_reviews)

[[ 6 21  0  0]
 [ 3  2  0  0]
 [23 13  0  0]
 [21  9 12  0]
 [ 4 16 17  0]
 [ 7 21  0  0]
 [21 16 13  0]
 [27  7  0  0]
 [27 23  0  0]
 [20 19  0  0]]


In [26]:
embeded_vector_size = 5
# creating in model in a sequential way.
model = Sequential()
# adding my first layer using Embedding that i imported earlier.
# just assigning embeded_vector_size, input length as my max length, also a name="embedding" which i can use earlier.
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length,name="embedding"))
# flatten those embedding vectors
model.add(Flatten())
# then we add 1 neuron output layer with sigmoid activation function.
model.add(Dense(1, activation='sigmoid'))

In [27]:
X = padded_reviews # assigning my padded out to X
y = sentiment # this assigned above - sentiment = np.array([1,1,1,1,1,0,0,0,0,0]), first 5 are positive reviews and, next 5 are negetive reviews.

In [28]:
# im using loss='binary_crossentropy' because my out is either 1 or 0, ie, positive or negetive.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 embedding (Embedding)       (None, 4, 5)              150       
                                                                 
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 171 (684.00 Byte)
Trainable params: 171 (684.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# train the model
model.fit(X, y, epochs=50, verbose=0)

In [30]:
# evaluate the model
loss, accuracy = model.evaluate(X, y)
accuracy

1/1 [==============================] - 0s 216ms/step - loss: 0.6266 - accuracy: 1.0000


1.0

In [ ]:
# this is our embedded wording.
# we use name we assigned earlier.
# this is the weight of all vectors (features)
weights = model.get_layer('embedding').get_weights()[0]
weights

array([[ 0.02525621,  0.03310835, -0.04979291,  0.08472165, -0.07781053],
       [ 0.04140401,  0.04554696,  0.0142842 , -0.04178649, -0.00398065],
       [ 0.06319761,  0.05469593,  0.0916774 ,  0.0691436 ,  0.04260057],
       [-0.06141017, -0.08621411,  0.09558144,  0.0996749 ,  0.08924381],
       [-0.04635488, -0.05954704,  0.05853776,  0.06600622,  0.04873991],
       [-0.02964232,  0.03196828, -0.00283191, -0.0307832 , -0.01620314],
       [-0.08496669, -0.00986668,  0.04644088,  0.02344804,  0.09203819],
       [ 0.07469345, -0.05078626, -0.06861632, -0.0985084 , -0.04210273],
       [ 0.03263037, -0.04552471,  0.0490703 , -0.04830383,  0.04754186],
       [ 0.0839077 ,  0.01336703,  0.01359792,  0.00332644,  0.08739428],
       [ 0.03857725, -0.00233979, -0.03952513,  0.01684934, -0.00230075],
       [ 0.00371198,  0.02089468,  0.02805854, -0.02096107, -0.02942134],
       [-0.043928  , -0.0695344 ,  0.05290475, -0.03965294,  0.07815702],
       [ 0.0530896 ,  0.07526175,  0.0

In [ ]:
len(weights) # as my vocabilary size was 30, my weights is also 30.

30

In [32]:
weights[13]

array([ 0.0530896 ,  0.07526175,  0.06364192,  0.06548808, -0.00441356],
      dtype=float32)

In [33]:
weights[4]

array([-0.04635488, -0.05954704,  0.05853776,  0.06600622,  0.04873991],
      dtype=float32)

In [34]:
weights[16]

array([-0.00645382, -0.06732691, -0.04259013, -0.0171776 , -0.0064818 ],
      dtype=float32)